### Import everything

In [3]:
%load_ext autoreload
%autoreload 2
from run import main
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error

### Run everything

In [4]:
%time
result, data = main()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs
Start script


Process ForkPoolWorker-16:
Process ForkPoolWorker-14:
Process ForkPoolWorker-13:
Process ForkPoolWorker-11:
Process ForkPoolWorker-15:
Process ForkPoolWorker-10:
Process ForkPoolWorker-12:
Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Process ForkPoolWorker-9:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/kiru/anaconda3/envs/ada/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/kiru/anaconda3/envs/a

  File "/Users/kiru/anaconda3/envs/ada/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/Users/kiru/anaconda3/envs/ada/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/Users/kiru/anaconda3/envs/ada/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Users/kiru/anaconda3/envs/ada/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Users/kiru/anaconda3/envs/ada/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Users/kiru/anaconda3/envs/ada/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Users/kiru/anaconda3/envs/ada/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/Users/kiru/anaconda3/envs/

KeyboardInterrupt: 

#### Save to pickle

In [ ]:
def read_from_file():
    with open('data.p', 'rb') as f:
        x = pickle.load(f)
        return x

In [ ]:
#save_to_pickle((result, data))
result, data = read_from_file()

In [ ]:
len(result)

In [ ]:
first = list(map(lambda x: x[1], result))
first_max = min(first, key=lambda x: x.rmse)

In [ ]:
def find_best_models(results):
    min_models = []
    for m in range(len(results[0])):
        bla = []
        for r in results:
            bla.append(r[m])
        min_model = min(bla, key=lambda x: x.rmse)
        min_models.append(min_model)
    return min_models

In [ ]:
best_models = find_best_models(result)

In [ ]:
models = best_models
models

## Blend 'em together

In [ ]:
    def build_predictions_table(data, models):
        blending_data = np.zeros((len(data), len(models)))
        for i_d, x in enumerate(data):
            for i_m, model in enumerate(models):
                pred = model.algo.predict(str(x[0]), str(x[1]))
                blending_data[i_d, i_m] = pred.est
        return blending_data

In [ ]:
%time
x = build_predictions_table(data, models)
y = list(map(lambda x: x[2], data))

In [ ]:
print(x[:1])
print(y[:1])

In [ ]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0)
clf.fit(x, y) 
clf.coef_

In [ ]:
def blend_them_together(models):
    from sklearn.model_selection import cross_validate
    from sklearn.linear_model import Ridge
    a = Ridge(alpha=1)
    cv_results = cross_validate(a, x, y, cv=3,
                                scoring=('neg_mean_squared_error'),
                                return_train_score=True, return_estimator=True)

    test_scores = cv_results['test_score']
    estimators = cv_results['estimator']
    best = estimators[np.argmax(np.array(test_scores))]
    print(best.coef_)
    return best

In [ ]:
blender_model = blend_them_together(models)

In [ ]:
def predict_using_blending(models, trainer, x):
    pred_sum = 0
    input = np.zeros((len(models), ))
    for i, m in enumerate(models):
        prediction = m.algo.predict(str(x[0]), str(x[1])).est
        input[i]  = prediction
    return trainer.predict([input])

In [ ]:
def get_all_predictions(models, blending_model):
    blend_predictions = []
    for each in data:
        blend_pred = predict_using_blending(models, blending_model, each)
        blend_pred_round = int(round(blend_pred[0]))
        blend_predictions.append(blend_pred_round)
    return blend_predictions

In [ ]:
blending_pred = get_all_predictions(models, blender_model)
np.sqrt(mean_squared_error(blending_pred, y))